In [1]:
!wget https://raw.githubusercontent.com/m-bashari-m/vehicle-color-recognition/main/src/utils.py

--2022-05-26 22:01:25--  https://raw.githubusercontent.com/m-bashari-m/vehicle-color-recognition/main/src/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7080 (6.9K) [text/plain]
Saving to: ‘utils.py’

utils.py            100%[===================>]   6.91K  --.-KB/s    in 0s      

2022-05-26 22:01:25 (61.7 MB/s) - ‘utils.py’ saved [7080/7080]



In [3]:
from google.colab import drive
drive.mount('./drive')

Mounted at ./drive


In [25]:
import pandas as pd
import numpy as np
import os

import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub

from utils import ModelCreator, get_train_val_ds, ErrorAnalyzer, get_class_weight

In [15]:
BATCH_SIZE = 32
IMG_SIZE = (512, 512)
AUTOTUNE = tf.data.AUTOTUNE
N_CLASSES = 16
HUB_URL = 'https://tfhub.dev/google/bit/m-r50x3/1'

In [6]:
dataset_dir = os.path.join('drive', 'MyDrive', 'cars')
train_dir = os.path.join(dataset_dir, 'train')
val_dir = os.path.join(dataset_dir, 'val')

In [17]:
train_ds, val_ds = get_train_val_ds(train_dir, val_dir, batch_size=BATCH_SIZE)

Found 17043 files belonging to 16 classes.
Found 3495 files belonging to 16 classes.


In [8]:
classes, class_weight = get_class_weight()

In [9]:
for color, weight in zip(classes, class_weight.values()):
  print(color, "    \t=>", weight)

beige     	=> 17.94
black     	=> 13.8
blue     	=> 15.47
brown     	=> 17.92
cream     	=> 56.81
crimson     	=> 76.43
gold     	=> 56.81
green     	=> 21.2
grey     	=> 12.41
navy-blue     	=> 58.77
orange     	=> 22.37
red     	=> 16.39
silver     	=> 20.17
titanium     	=> 56.81
white     	=> 2.97
yellow     	=> 20.68


In [18]:
train_ds = (
    train_ds.
    map(lambda img, lbl: (img/255., lbl), num_parallel_calls=AUTOTUNE).
    prefetch(AUTOTUNE))

val_ds = (
    val_ds.
    map(lambda img, lbl: (img/255., lbl), num_parallel_calls=AUTOTUNE).
    prefetch(AUTOTUNE))

In [21]:
rgb_creator = ModelCreator(HUB_URL, 'RGB-model')

In [27]:
rgb_model = rgb_creator.make_model()
callbacks = rgb_creator.get_callbacks()

In [23]:
rgb_model.summary()

Model: "RGB-model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_1 (KerasLayer)  (None, 6144)              211174080 
                                                                 
 dense_4 (Dense)             (None, 600)               3687000   
                                                                 
 dropout_2 (Dropout)         (None, 600)               0         
                                                                 
 dense_5 (Dense)             (None, 16)                9616      
                                                                 
Total params: 214,870,696
Trainable params: 3,696,616
Non-trainable params: 211,174,080
_________________________________________________________________


In [28]:
history = rgb_model.fit(train_ds,
                        callbacks=callbacks,
                        epochs=10,
                        steps_per_epoch=1)

Epoch 1/10
1/1 [==============================] - ETA: 0s - loss: 6.3775 - precision: 0.2000 - recall: 0.1719 - auc: 0.1250 - accuracy: 0.4551

KeyError: ignored

In [ ]:
img, lbl = next(iter(train_ds))

In [ ]:
img.shape, lbl.shape